In [1]:
import os
import time
import math
import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import mode
from sklearn.cluster import KMeans
import pandas as pd

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import datetime
from datetime import datetime as dt

import random

from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense, Input, BatchNormalization, Dropout, Conv2D, Flatten

In [2]:
filepath = './horizontal/Video-15-47-49-515-31-03-2022/Video-15-47-49-515 (4IIC).mp4'
filepath_check = './checkpoint/'
input_dim = 500
maxCorners = 100
frames = 30
net_len = 1000
units = 100
output_size = 2
classes_list = ["stop", "go"]

In [3]:
horizontal = ['./horizontal/Video-7-36-30-868-22-03-2022/Video-7-36-30-868 (S1WK)',
            #'./horizontal/Video-10-6-45-47-30-03-2022/Video-10-6-45-47 (J57V)',
            './horizontal/Video-15-43-13-884-22-03-2022/Video-15-43-13-884 (J4NP)',
            './horizontal/Video-10-25-45-905-31-03-2022/Video-10-25-45-905 (3W3K)',
            './horizontal/Video-10-56-13-801-31-03-2022/Video-10-56-13-801 (X0N9)',            
            './horizontal/Video-11-19-18-132-30-03-2022/Video-11-19-18-132 (SH2G)',
            './horizontal/Video-12-51-53-673-27-03-2021/Video-12-51-53-673 (NKXJ)',
            #'./horizontal/Video-15-26-20-118-29-03-2022/Video-15-26-20-118 (5FLZ)',
            './horizontal/Video-15-32-33-709-31-03-2022/Video-15-32-33-709 (7BAA)',
            './horizontal/Video-15-41-30-94-29-03-2022/Video-15-41-30-94 (PH33)',
            #'./horizontal/Video-15-47-49-515-31-03-2022/Video-15-47-49-515 (4IIC)',
            './horizontal/Video-15-49-30-274-29-03-2022/Video-15-49-30-274 (JP92)',
            './horizontal/Video-15-8-58-789-27-03-2022/Video-15-8-58-789 (SSQM)',
            './horizontal/Video-17-14-15-118-06-03-2021/Video-17-14-15-118 (PE9P)',
            './horizontal/Video-17-34-45-652-25-03-2021/Video-17-34-45-652 (BK89)',
            './horizontal/Video-17-44-40-74-06-03-2021/Video-17-44-40-74 (CLNF)',
            './horizontal/Video-7-51-44-763-22-03-2022/Video-7-51-44-763 (1QN6)',
            './horizontal/Video-8-6-54-707-22-03-2022/Video-8-6-54-707 (PKAY)']

vertical = ['./vertical/Video-10-15-12-812-14-11-2020/Video-10-15-12-812 (2Y5M)',
           './vertical/Video-10-30-21-575-29-09-2021/Video-10-30-21-575 (UOM8)',
           './vertical/Video-11-21-2-419-13-09-2021/Video-11-21-2-419 (KVYG)',
           './vertical/Video-11-21-6-412-28-12-2020/Video-11-21-6-412 (JQQM)',
           './vertical/Video-11-36-12-363-28-09-2021/Video-11-36-12-363 (8119)',
           './vertical/Video-11-36-16-421-28-12-2020/Video-11-36-16-421 (PFAQ)',
           './vertical/Video-11-5-49-336-13-09-2021/Video-11-5-49-336 (CZPL)',
           './vertical/Video-12-23-51-640-02-10-2021/Video-12-23-51-640 (WLWK)',
           './vertical/Video-12-39-1-595-02-10-2021/Video-12-39-1-595 (QDUT)',
           './vertical/Video-12-44-36-682-24-09-2021/Video-12-44-36-682 (ATO6)',
           './vertical/Video-13-16-32-822-10-09-2021/Video-13-16-32-822 (UQ1Q)',
           './vertical/Video-13-20-30-695-15-11-2021/Video-13-20-30-695 (VLFO)',
           './vertical/Video-13-31-46-773-10-09-2021/Video-13-31-46-773 (MXM7)',
           './vertical/Video-16-16-44-458-17-11-2020/Video-16-16-44-458 (LCRL)',
           './vertical/Video-16-36-42-174-10-11-2020/Video-16-36-42-174 (IXTE)',
           './vertical/Video-16-47-10-870-17-11-2020/Video-16-47-10-870 (YFCN)',
           './vertical/Video-17-17-30-919-17-11-2020/Video-17-17-30-919 (ID6X)',
           './vertical/Video-17-32-40-752-17-11-2020/Video-17-32-40-752 (E575)',
           './vertical/Video-17-47-40-713-02-10-2021/Video-17-47-40-713 (E7IK)',
           './vertical/Video-20-21-59-727-24-09-2021/Video-20-21-59-727 (S04R)',
           './vertical/Video-20-45-56-152-24-10-2021/Video-20-45-56-152 (6M38)']

random.shuffle(horizontal)
random.shuffle(vertical)

videopath = horizontal# + vertical

random.shuffle(videopath)

In [4]:
def six_or_else(a):
    if a[1]==6:
        return [1,a[2]]
    else:
        return [0,a[2]]

def mode_periods(df):
    last_mode = -1
    mode_time = []
    mode_time.append([1 if df.iloc[0][0] == 6 else 0 , '00:00:00.0000000']) 
    for row in df.itertuples():  
        if row[1] != last_mode:   
            mode_time.append(six_or_else(row))
        last_mode = row[1]
    mode_time.append([1 if df.iloc[-1][0] == 6 else 0 , '1'+df.iloc[-1][1][1:] ] ) 
    
    return mode_time

In [5]:
def data(filepath):
    df = pd.read_csv(filepath[:-4]+'.csv')[['activityMode','timestamp']]
    mode_time = mode_periods(df)
    
    # Read the video 
    cap = cv2.VideoCapture(filepath)
    cap.set(cv2.CAP_PROP_BUFFERSIZE, 2)
    
    # Parameters for ShiTomasi corner detection
    feature_params = dict(maxCorners = maxCorners,
                       qualityLevel = 0.07,
                       minDistance = 10,
                       blockSize = 5 )

    # Parameters for Lucas Kanade optical flow
    lk_params = dict(winSize  = (15, 15),
                  maxLevel = 2,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))
    net = []
    label_list = []
    i_frame = 0
    j_frame = 0
    while True:       
        ret, old_frame = cap.read()
        if not ret:
                break
        old_frame = cv2.resize(old_frame, (input_dim, input_dim))
        height, width, _ = old_frame.shape 
        part_width = round(width * 0.4)
        part_height = round(height * 0.4)
        
        old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
        
        p0 = cv2.goodFeaturesToTrack(old_gray, mask=None, **feature_params)
        cv2.rectangle(old_gray, (part_width, part_height), 
                          (width - part_width, height - part_height), 0, -1)
        # Create a mask image for drawing purposes
        mask = np.zeros_like(old_frame)
        
        while True:
            # Read new frame
            ret, frame = cap.read()
            
            if not ret:
                break
                
            sec = cap.get(cv2.CAP_PROP_POS_MSEC)
            res = datetime.timedelta(seconds = sec/1000)
            for i in range(len(mode_time)-1): 
                if (dt.strptime(mode_time[i][1][:-8], "%H:%M:%S") - datetime.datetime(1900, 1, 1) <= res) and\
                    (dt.strptime(mode_time[i+1][1][:-8], "%H:%M:%S") - datetime.datetime(1900, 1, 1) >= res):
                    label = [mode_time[i][0]]                
                    break  
                    
            if (len(label_list)!=0)and(label == label_list[-1])and(j_frame>maxCorners):
                continue 
                
            frame = cv2.resize(frame, (input_dim, input_dim))
            frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            cv2.rectangle(old_gray, (part_width, part_height), 
                          (width - part_width, height - part_height), 0, -1)
            # Calculate Optical Flow
            p1, st, err = cv2.calcOpticalFlowPyrLK(
                old_gray, frame_gray, p0, None, **lk_params
            )

            # Select good points
            good_new = p1[st == 1]
            good_old = p0[st == 1]
            
            new_net = good_new - good_old
            #print(new_net)
            while new_net.shape[0] < maxCorners:                
                new_net = np.append(new_net, [[0,0]], axis=0)                  
            new_net = np.array(new_net)
            new_net[np.lexsort((new_net[:,1],new_net[:,0]))]
            
            i_frame += 1
            
            if True:#(len(label_list)==0):
                j_frame += 1
                net.append(new_net) 
                label_list.append(label)
                # Update the previous frame and previous points
                old_gray = frame_gray.copy()
                p0 = good_new.reshape(-1, 1, 2)
                
                #if j_frame >= 100:
                   # break
                continue
            
            if ((label == label_list[-1]) and (j_frame<=frames)):
                j_frame += 1
                net.append(new_net) 
                label_list.append(label)
                # Update the previous frame and previous points
                old_gray = frame_gray.copy()
                p0 = good_new.reshape(-1, 1, 2)
                if j_frame >= frames:
                    break
                continue           
                        
            if (label != label_list[-1]):
                j_frame = 0
                net.append(new_net) 
                label_list.append(label)
                # Update the previous frame and previous points
                old_gray = frame_gray.copy()
                p0 = good_new.reshape(-1, 1, 2)
                break
                
    return np.array(net)/input_dim, np.array(label_list)

In [6]:
def data_full(filepath):
    df = pd.read_csv(filepath[:-4]+'.csv')[['activityMode','timestamp']]
    mode_time = mode_periods(df)
    
    # Read the video 
    cap = cv2.VideoCapture(filepath)
    cap.set(cv2.CAP_PROP_BUFFERSIZE, 2)
    
    # Parameters for ShiTomasi corner detection
    feature_params = dict(maxCorners = maxCorners,
                       qualityLevel = 0.07,
                       minDistance = 10,
                       blockSize = 5 )

    # Parameters for Lucas Kanade optical flow
    lk_params = dict(winSize  = (15, 15),
                  maxLevel = 2,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))
    net = []
    label_list = []
    while True:       
        ret, old_frame = cap.read()
        
        if not ret:
                break
                
        old_frame = cv2.resize(old_frame, (input_dim, input_dim))
        height, width, _ = old_frame.shape 
        part_width = round(width * 0.4)
        part_height = round(height * 0.4)
        
        old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
        
        p0 = cv2.goodFeaturesToTrack(old_gray, mask=None, **feature_params)
        #cv2.rectangle(old_gray, (part_width, part_height), 
                          #(width - part_width, height - part_height), 0, -1)
        # Create a mask image for drawing purposes
        mask = np.zeros_like(old_frame)
        
        while True:
            # Read new frame
            ret, frame = cap.read()
            
            if not ret:
                break
                
            sec = cap.get(cv2.CAP_PROP_POS_MSEC)
            res = datetime.timedelta(seconds = sec/1000)
            for i in range(len(mode_time)-1): 
                if (dt.strptime(mode_time[i][1][:-8], "%H:%M:%S") - datetime.datetime(1900, 1, 1) <= res) and\
                    (dt.strptime(mode_time[i+1][1][:-8], "%H:%M:%S") - datetime.datetime(1900, 1, 1) >= res):
                    label = [mode_time[i][0]]                
                    break                    
                
            frame = cv2.resize(frame, (input_dim, input_dim))
            frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            #cv2.rectangle(old_gray, (part_width, part_height), 
                          #(width - part_width, height - part_height), 0, -1)
            # Calculate Optical Flow
            p1, st, err = cv2.calcOpticalFlowPyrLK(
                old_gray, frame_gray, p0, None, **lk_params
            )
            if p1 is None:
                break
            # Select good points
            good_new = p1[st == 1]
            good_old = p0[st == 1]
            
            new_net = good_new - good_old
            
            while new_net.shape[0] < maxCorners:                
                new_net = np.append(new_net, [[-999999,-999999]], axis=0) 
            #new_net = [[x[0] + input_dim,x[1] + input_dim] for x in new_net]
            #new_net = np.array(new_net)/(2*input_dim)
            new_net[np.lexsort((new_net[:,1],new_net[:,0]))]
            

            old_gray = frame_gray.copy()
            p0 = good_new.reshape(-1, 1, 2)
            
            net.append(new_net)
            label_list.append(label)
                
    return np.array(net), np.array(label_list)

In [43]:
def lucas_kanade_method_orig(video_path, callbacks, val_x, val_y):
    df = pd.read_csv(video_path[:-4]+'.csv')[['activityMode','timestamp']]
    mode_time = mode_periods(df)
    
    model = get_CNN(units, input_dim, output_size)
    
    model.compile(
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        optimizer="sgd",
        metrics=["accuracy"]
    )
    
    # Read the video 
    cap = cv2.VideoCapture(video_path)
    cap.set(cv2.CAP_PROP_BUFFERSIZE, 2)
    
    # Parameters for ShiTomasi corner detection
    feature_params = dict(maxCorners = maxCorners,
                       qualityLevel = 0.07,
                       minDistance = 7,
                       blockSize = 7 )

    # Parameters for Lucas Kanade optical flow
    lk_params = dict(winSize  = (15, 15),
                  maxLevel = 3,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

    # Create random colors
    color = np.random.randint(0, 255, (500, 3))
    predicted_class_name = ' '
    while True:       
        start = time.time()
        # Take first frame and find corners in it
        ret, old_frame = cap.read()
        old_frame = cv2.resize(old_frame, (input_dim, input_dim))
        height, width, _ = old_frame.shape 
        part_width = round(width * 0.3)
        part_height = round(height * 0.3)
        
        old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
        
        #cv2.rectangle(old_gray, (part_width, part_height), 
                          #(width - part_width, height - part_height), 0, -1)
        
        p0 = cv2.goodFeaturesToTrack(old_gray, mask=None, **feature_params)

        # Create a mask image for drawing purposes
        mask = np.zeros_like(old_frame)
        net = []
        label_list = []
        
        while True:
            # Read new frame
            ret, frame = cap.read()
            frame = cv2.resize(frame, (input_dim, input_dim))
            if not ret:
                break

            #cv2.rectangle(frame, (part_width, part_height), 
                          #(width - part_width, height - part_height), 0, -1)
            
            frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            # Calculate Optical Flow
            p1, st, err = cv2.calcOpticalFlowPyrLK(
                old_gray, frame_gray, p0, None, **lk_params
            )

            # Select good points
            good_new = p1[st == 1]
            good_old = p0[st == 1]
            
            new_net = good_new - good_old
            #print(new_net)
            while new_net.shape[0] < maxCorners:                
                new_net = np.append(new_net, [[-999999,-999999]], axis=0)                  
            new_net = np.array(new_net)
            new_net[np.lexsort((new_net[:,1],new_net[:,0]))]   
            net.append(new_net)
            
            # Draw the tracks
            for i, (new, old) in enumerate(zip(good_new, good_old)):
                a, b = new.ravel()
                c, d = old.ravel()
                mask = cv2.line(mask, (int(a), int(b)), (int(c), int(d)), color[i].tolist(), 2)
                frame = cv2.circle(frame, (int(a), int(b)), 5, color[i].tolist(), -1)

            sec = cap.get(cv2.CAP_PROP_POS_MSEC)
            res = datetime.timedelta(seconds = sec/1000)
            for i in range(len(mode_time)-1): 
                if (dt.strptime(mode_time[i][1][:-8], "%H:%M:%S") - datetime.datetime(1900, 1, 1) <= res) and\
                    (dt.strptime(mode_time[i+1][1][:-8], "%H:%M:%S") - datetime.datetime(1900, 1, 1) >= res):
                    label = [mode_time[i][0]]                
                    break  
            
            label_list.append(label)                
                
            # Display the demo
            img = cv2.add(frame, mask)
            cv2.putText(img, predicted_class_name, 
                        (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            cv2.imshow("frame", img)
            k = cv2.waitKey(25) & 0xFF
            if k == 27:
                break

            # Update the previous frame and previous points
            old_gray = frame_gray.copy()
            p0 = good_new.reshape(-1, 1, 2)
            if np.array(net).shape[0] >= frames:
                net = np.array(net) / input_dim
                #history = model.fit(
                    #net, np.array(label_list), batch_size=20, epochs=1,
                    #callbacks=callbacks,
                    #validation_data  = (val_x, val_y) 
                #)
                predicted_labels_probabilities = model.predict(net)
                summa = predicted_labels_probabilities[0]
                print(predicted_labels_probabilities)
                for i in range(1, len(predicted_labels_probabilities)):
                    summa += predicted_labels_probabilities[i]
                #print(model.predict(net))
                predicted_label = np.argmax(summa)
        
                # Accessing The Class Name using predicted label.
                predicted_class_name = classes_list[predicted_label]
                
                break

In [8]:
val_x, val_y = data_full('./horizontal/Video-10-6-45-47-30-03-2022/Video-10-6-45-47 (J57V).mp4')
print(val_x.shape, val_y.shape)

(26707, 100, 2) (26707, 1)


In [44]:
checkpoint = keras.callbacks.ModelCheckpoint(
        filepath_check, monitor="val_accuracy",
        save_best_only=True, verbose=1,
        mode='max')
callbacks = [checkpoint]

In [10]:
def get_RNN(units, input_dim, output_size):
    model = keras.Sequential()
    model.add(layers.LSTM(units, return_sequences=True, input_shape=(None, 2)))
    model.add(layers.GRU(64))
    model.add(layers.Dropout(0.4))
    model.add(layers.Dense(32, activation="relu"))
    model.add(layers.Dense(output_size))

    return model

In [37]:
def get_CNN(units, input_dim, output_size): 
    model = keras.Sequential()
    #model.add(Conv2D(2, (2, 2), input_shape=(100, 2, 1)))
    model.add(Input((maxCorners,2)))
    model.add(Flatten())  
    #model.add(Dense(500, activation='relu')) 
    #model.add(Dropout(0.4))
    #model.add(BatchNormalization())
    #model.add(Dense(250, activation='relu'))
    #model.add(Dropout(0.4))
    #model.add(BatchNormalization())
    #model.add(Dense(100, activation='relu'))
    #model.add(Dropout(0.4))
    #model.add(BatchNormalization())
    model.add(Dense(55, activation='relu'))
    model.add(Dropout(0.4))
    model.add(BatchNormalization())
    model.add(Dense(2, activation='softmax'))
    
    
    return model

In [37]:
val_x_f, val_y_f = data_full('./horizontal/Video-15-47-49-515-31-03-2022/Video-15-47-49-515 (4IIC).mp4')
print(val_x_f.shape, val_y_f.shape)

(101, 100, 2) (101, 1)


In [12]:
# Adding Early Stopping Callback
early_stopping_callback = EarlyStopping(monitor = 'val_accuracy', patience = 15,
                                        mode = 'max', restore_best_weights = True)

model = get_CNN(units, input_dim, output_size)

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
                   metrics=['accuracy'])

2022-05-13 19:08:03.704868: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-05-13 19:08:03.704917: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (alexander-HP-Pavilion-Gaming-Laptop-15-cx0xxx): /proc/driver/nvidia/version does not exist
2022-05-13 19:08:03.706997: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [13]:
xt, yt = data_full('./horizontal/Video-15-26-20-118-29-03-2022/Video-15-26-20-118 (5FLZ).mp4')

In [ ]:
xt2, yt2 = data_full('./horizontal/Video-12-51-53-673-27-03-2021/Video-12-51-53-673 (NKXJ).mp4')

In [40]:
model.fit(
            xt, yt, batch_size=20, epochs=5,
            callbacks = callbacks,
            validation_data  = (val_x, val_y),
            shuffle = True
    
        )

model.fit(
            xt2, yt2, batch_size=20, epochs=5,
            callbacks = callbacks,
            validation_data  = (val_x, val_y),
            shuffle = True
    
        )

Epoch 1/5
 617/1322 [=============>................] - ETA: 1s - loss: 0.0143 - accuracy: 1.0000

KeyboardInterrupt: 

In [22]:
xx = np.concatenate((xt2, xt), 0)
yy = np.concatenate((yt2, yt), 0)

In [41]:
model.fit(
            xx, yy, batch_size=20, epochs=15,
            callbacks = callbacks,
            validation_data  = (val_x, val_y),
            shuffle = True
    
        )

Epoch 1/15
1933/1938 [============================>.] - ETA: 0s - loss: 0.6488 - accuracy: 0.6821
Epoch 1: val_accuracy improved from -inf to 0.67694, saving model to ./checkpoint/
INFO:tensorflow:Assets written to: ./checkpoint/assets
1938/1938 [==============================] - 6s 3ms/step - loss: 0.6487 - accuracy: 0.6821 - val_loss: 1643.5432 - val_accuracy: 0.6769
Epoch 2/15
1909/1938 [============================>.] - ETA: 0s - loss: 0.6263 - accuracy: 0.6820
Epoch 2: val_accuracy improved from 0.67694 to 0.75074, saving model to ./checkpoint/
INFO:tensorflow:Assets written to: ./checkpoint/assets
1938/1938 [==============================] - 5s 3ms/step - loss: 0.6261 - accuracy: 0.6821 - val_loss: 7379.3228 - val_accuracy: 0.7507
Epoch 3/15
1920/1938 [============================>.] - ETA: 0s - loss: 0.6262 - accuracy: 0.6822
Epoch 3: val_accuracy improved from 0.75074 to 0.75295, saving model to ./checkpoint/
INFO:tensorflow:Assets written to: ./checkpoint/assets
1938/1938 [===

In [47]:
lucas_kanade_method_orig('./horizontal/Video-15-43-13-884-22-03-2022/Video-15-43-13-884 (J4NP).mp4',
                         callbacks, val_x, val_x)

[[0.50000006 0.4999999 ]
 [0.500001   0.49999896]
 [0.49999306 0.5000069 ]
 [0.49999803 0.50000197]
 [0.50000757 0.49999243]
 [0.50000244 0.49999762]
 [0.49999633 0.50000364]
 [0.4999965  0.5000035 ]
 [0.50000304 0.49999702]
 [0.49999574 0.50000423]
 [0.4999967  0.5000033 ]
 [0.49999684 0.5000032 ]
 [0.50000197 0.49999806]
 [0.5000011  0.4999989 ]
 [0.50000125 0.49999875]
 [0.49999705 0.500003  ]
 [0.5000035  0.4999965 ]
 [0.50000274 0.49999732]
 [0.4999941  0.50000596]
 [0.5000003  0.49999973]
 [0.500004   0.49999604]
 [0.49999937 0.5000006 ]
 [0.5000011  0.4999989 ]
 [0.50001913 0.49998087]
 [0.5000062  0.49999383]
 [0.49999398 0.500006  ]
 [0.49999005 0.50000995]
 [0.50000054 0.49999943]
 [0.4999925  0.50000757]
 [0.49998882 0.50001115]]
[[0.4999994  0.5000006 ]
 [0.49999574 0.50000423]
 [0.50000787 0.4999922 ]
 [0.49999675 0.5000033 ]
 [0.49999312 0.50000685]
 [0.49999958 0.5000005 ]
 [0.49999702 0.50000304]
 [0.49999776 0.5000022 ]
 [0.49999917 0.50000083]
 [0.5000042  0.49999583]

[[0.4999995  0.50000054]
 [0.50000626 0.4999938 ]
 [0.5000005  0.49999958]
 [0.500002   0.49999794]
 [0.49999803 0.50000197]
 [0.5000045  0.4999955 ]
 [0.5000005  0.4999995 ]
 [0.50000286 0.49999717]
 [0.4999983  0.50000167]
 [0.5000002  0.4999998 ]
 [0.49999782 0.5000022 ]
 [0.4999992  0.50000083]
 [0.4999997  0.5000003 ]
 [0.49999514 0.5000049 ]
 [0.5000022  0.4999978 ]
 [0.49999648 0.5000035 ]
 [0.49999937 0.50000066]
 [0.49999824 0.5000017 ]
 [0.4999859  0.50001407]
 [0.49999613 0.50000393]
 [0.49999604 0.50000393]
 [0.49999744 0.5000026 ]
 [0.49999732 0.5000027 ]
 [0.50000507 0.49999496]
 [0.5000029  0.4999971 ]
 [0.5000028  0.49999717]
 [0.49999908 0.5000009 ]
 [0.49999863 0.5000014 ]
 [0.49999136 0.50000864]
 [0.5000015  0.4999985 ]]
[[0.5000016  0.49999836]
 [0.49999297 0.50000703]
 [0.49999902 0.500001  ]
 [0.5000045  0.4999955 ]
 [0.49999878 0.50000125]
 [0.5000004  0.49999964]
 [0.50000286 0.49999717]
 [0.49999318 0.50000685]
 [0.5000006  0.49999946]
 [0.5000009  0.49999908]

[[0.50000423 0.49999583]
 [0.500001   0.49999896]
 [0.5000046  0.49999547]
 [0.499997   0.50000304]
 [0.5000009  0.49999908]
 [0.49999782 0.50000215]
 [0.50000244 0.49999762]
 [0.5000035  0.4999965 ]
 [0.5000012  0.49999887]
 [0.49999705 0.500003  ]
 [0.50000024 0.49999982]
 [0.5000011  0.4999989 ]
 [0.4999989  0.5000011 ]
 [0.49999946 0.5000006 ]
 [0.5000007  0.4999993 ]
 [0.5000039  0.49999613]
 [0.5000005  0.49999952]
 [0.50000376 0.49999622]
 [0.500006   0.49999395]
 [0.4999957  0.5000043 ]
 [0.5000009  0.49999908]
 [0.4999989  0.5000011 ]
 [0.49999943 0.5000006 ]
 [0.50000364 0.49999636]
 [0.5000023  0.49999768]
 [0.49999642 0.5000036 ]
 [0.4999963  0.50000376]
 [0.49999586 0.5000041 ]
 [0.5000005  0.49999952]
 [0.5000032  0.49999678]]
[[0.49999967 0.5000003 ]
 [0.5000011  0.49999893]
 [0.49999753 0.50000244]
 [0.49999514 0.5000048 ]
 [0.49999955 0.5000005 ]
 [0.5000024  0.4999976 ]
 [0.49998605 0.50001395]
 [0.50000024 0.4999998 ]
 [0.4999963  0.5000037 ]
 [0.49999514 0.5000048 ]

[[0.5003432  0.49965683]
 [0.5001528  0.49984714]
 [0.4999298  0.50007015]
 [0.4999813  0.5000187 ]
 [0.49986205 0.500138  ]
 [0.49983153 0.5001685 ]
 [0.5001003  0.4998997 ]
 [0.5000838  0.4999162 ]
 [0.50005794 0.49994203]
 [0.5000654  0.49993458]
 [0.50053954 0.4994605 ]
 [0.50177515 0.49822485]
 [0.50265473 0.49734527]
 [0.49947652 0.50052345]
 [0.49771944 0.5022806 ]
 [0.49965242 0.5003476 ]
 [0.4989703  0.50102973]
 [0.49993727 0.5000627 ]
 [0.49972296 0.50027704]
 [0.4998019  0.5001981 ]
 [0.49924594 0.50075406]
 [0.49985504 0.500145  ]
 [0.49899852 0.50100154]
 [0.4986816  0.50131834]
 [0.49863866 0.50136137]
 [0.49867883 0.50132114]
 [0.49868348 0.5013165 ]
 [0.4985576  0.50144243]
 [0.498621   0.5013791 ]
 [0.4987848  0.5012151 ]]
[[0.49972713 0.50027287]
 [0.49970645 0.5002935 ]
 [0.4996321  0.50036794]
 [0.49973246 0.5002675 ]
 [0.49977946 0.50022054]
 [0.5000004  0.49999955]
 [0.49970728 0.5002927 ]
 [0.4995161  0.5004839 ]
 [0.49949735 0.5005026 ]
 [0.4995161  0.50048393]

[[0.49999353 0.50000644]
 [0.5000034  0.4999966 ]
 [0.5000016  0.49999845]
 [0.5000025  0.4999975 ]
 [0.49999252 0.5000075 ]
 [0.500002   0.49999794]
 [0.5000026  0.4999974 ]
 [0.4999983  0.50000167]
 [0.5000044  0.49999562]
 [0.5000027  0.49999738]
 [0.50000155 0.49999842]
 [0.5000023  0.49999768]
 [0.49999797 0.500002  ]
 [0.5000007  0.4999993 ]
 [0.49999887 0.5000012 ]
 [0.5000003  0.4999997 ]
 [0.50000083 0.49999923]
 [0.5000022  0.49999776]
 [0.49999726 0.50000274]
 [0.49999702 0.50000304]
 [0.49999937 0.5000006 ]
 [0.4999975  0.5000025 ]
 [0.4999971  0.50000286]
 [0.500002   0.49999803]
 [0.49999967 0.5000003 ]
 [0.5000028  0.49999717]
 [0.49999645 0.5000036 ]
 [0.49999973 0.50000024]
 [0.49999693 0.50000304]
 [0.499999   0.5000011 ]]
[[0.49999875 0.50000125]
 [0.5000053  0.4999947 ]
 [0.49999464 0.5000054 ]
 [0.5000025  0.4999975 ]
 [0.5000001  0.49999988]
 [0.4999967  0.5000033 ]
 [0.5000003  0.4999997 ]
 [0.49999943 0.5000006 ]
 [0.500003   0.49999702]
 [0.5000019  0.49999815]

...

[[0.4999763  0.50002366]
 [0.50032973 0.49967033]
 [0.5002924  0.4997076 ]
 [0.50038826 0.49961174]
 [0.49999744 0.50000256]
 [0.49999857 0.5000015 ]
 [0.49995458 0.5000455 ]
 [0.49994162 0.50005835]
 [0.49994802 0.500052  ]
 [0.49998385 0.5000162 ]
 [0.50002325 0.49997675]
 [0.49998507 0.50001496]
 [0.49998507 0.5000149 ]
 [0.49997914 0.50002086]
 [0.49997878 0.5000213 ]
 [0.5000203  0.49997973]
 [0.49997956 0.5000205 ]
 [0.4999959  0.5000042 ]
 [0.49997756 0.5000225 ]
 [0.49997896 0.50002104]
 [0.50008714 0.49991286]
 [0.50004345 0.49995652]
 [0.49999917 0.5000009 ]
 [0.5000021  0.49999788]
 [0.4999653  0.5000347 ]
 [0.49999598 0.50000405]
 [0.5000034  0.49999663]
 [0.4999366  0.50006336]
 [0.49999902 0.50000095]
 [0.5000025  0.49999756]]
[[0.5000163  0.4999838 ]
 [0.4999926  0.50000745]
 [0.50000805 0.49999198]
 [0.5000256  0.49997446]
 [0.49997687 0.5000232 ]
 [0.5000205  0.4999795 ]
 [0.50001824 0.49998176]
 [0.49999413 0.50000584]
 [0.50000495 0.49999508]
 [0.4999979  0.5000021 ]

[[0.5007351  0.49926487]
 [0.5002477  0.49975234]
 [0.50031585 0.49968418]
 [0.5007389  0.49926108]
 [0.50075835 0.49924162]
 [0.5006851  0.49931496]
 [0.500476   0.49952406]
 [0.50103945 0.4989605 ]
 [0.50086427 0.49913576]
 [0.50082994 0.49917006]
 [0.5007631  0.49923697]
 [0.50061136 0.4993887 ]
 [0.5006569  0.499343  ]
 [0.5010946  0.49890545]
 [0.50062937 0.49937063]
 [0.50090146 0.49909854]
 [0.5008111  0.4991889 ]
 [0.5003009  0.49969915]
 [0.5004222  0.49957782]
 [0.4999422  0.50005776]
 [0.5001752  0.49982488]
 [0.49971443 0.5002856 ]
 [0.49984312 0.50015694]
 [0.50012827 0.4998718 ]
 [0.5006016  0.49939844]
 [0.5003977  0.49960235]
 [0.5013984  0.49860153]
 [0.5010088  0.49899122]
 [0.50083804 0.49916202]
 [1.         0.        ]]
[[0.50109947 0.49890053]
 [0.5011207  0.4988793 ]
 [0.5011474  0.4988526 ]
 [0.50111115 0.49888882]
 [0.50100505 0.49899498]
 [0.50087845 0.49912158]
 [0.50084716 0.4991529 ]
 [0.5008422  0.49915785]
 [0.5007527  0.49924728]
 [0.50101304 0.49898693]

[[5.0251371e-01 4.9748626e-01]
 [5.0294739e-01 4.9705261e-01]
 [5.0257242e-01 4.9742761e-01]
 [5.0374186e-01 4.9625817e-01]
 [1.0000000e+00 0.0000000e+00]
 [1.0000000e+00 0.0000000e+00]
 [1.0000000e+00 0.0000000e+00]
 [1.0000000e+00 0.0000000e+00]
 [1.0000000e+00 0.0000000e+00]
 [1.0000000e+00 0.0000000e+00]
 [1.0000000e+00 0.0000000e+00]
 [1.0000000e+00 0.0000000e+00]
 [1.0000000e+00 0.0000000e+00]
 [1.0000000e+00 0.0000000e+00]
 [1.0000000e+00 0.0000000e+00]
 [1.0000000e+00 0.0000000e+00]
 [1.0000000e+00 0.0000000e+00]
 [1.0000000e+00 0.0000000e+00]
 [1.0000000e+00 0.0000000e+00]
 [1.0000000e+00 0.0000000e+00]
 [1.0000000e+00 0.0000000e+00]
 [1.0000000e+00 0.0000000e+00]
 [9.9936074e-01 6.3921546e-04]
 [9.9936050e-01 6.3948851e-04]
 [9.9936229e-01 6.3769671e-04]
 [9.9935848e-01 6.4155919e-04]
 [9.9936002e-01 6.4002507e-04]
 [9.9936229e-01 6.3773565e-04]
 [9.9936050e-01 6.3947873e-04]
 [9.9936098e-01 6.3906936e-04]]
[[5.01140237e-01 4.98859763e-01]
 [5.00549853e-01 4.99450117e-01]
 [5

[[4.9984789e-01 5.0015211e-01]
 [4.9984169e-01 5.0015837e-01]
 [1.0000000e+00 0.0000000e+00]
 [1.0000000e+00 0.0000000e+00]
 [1.0000000e+00 0.0000000e+00]
 [1.0000000e+00 0.0000000e+00]
 [1.0000000e+00 0.0000000e+00]
 [1.0000000e+00 0.0000000e+00]
 [1.0000000e+00 0.0000000e+00]
 [1.0000000e+00 0.0000000e+00]
 [1.0000000e+00 1.1326469e-23]
 [1.0000000e+00 0.0000000e+00]
 [0.0000000e+00 1.0000000e+00]
 [0.0000000e+00 1.0000000e+00]
 [0.0000000e+00 1.0000000e+00]
 [0.0000000e+00 1.0000000e+00]
 [0.0000000e+00 1.0000000e+00]
 [0.0000000e+00 1.0000000e+00]
 [0.0000000e+00 1.0000000e+00]
 [0.0000000e+00 1.0000000e+00]
 [0.0000000e+00 1.0000000e+00]
 [0.0000000e+00 1.0000000e+00]
 [0.0000000e+00 1.0000000e+00]
 [0.0000000e+00 1.0000000e+00]
 [0.0000000e+00 1.0000000e+00]
 [0.0000000e+00 1.0000000e+00]
 [0.0000000e+00 1.0000000e+00]
 [0.0000000e+00 1.0000000e+00]
 [0.0000000e+00 1.0000000e+00]
 [0.0000000e+00 1.0000000e+00]]
[[5.0040597e-01 4.9959412e-01]
 [5.0107747e-01 4.9892250e-01]
 [1.000

KeyboardInterrupt: 

In [15]:
for i in videopath:        
    x, y = data_full(i + '.mp4')   
      
    model.fit(
            np.array(x), np.array(y), batch_size=20, epochs=3,
            callbacks = callbacks,
            validation_data  = (val_x, val_y),
            shuffle = True 
    )

KeyboardInterrupt: 